# GDAL

In [1]:
from osgeo import gdal, ogr


## Point extractor

This script is intended to extracting values of raster from points.

In [10]:
def extract_values_from_raster(raster_path, points_path):
    # Open raster dataset
    raster_ds = gdal.Open(raster_path)

    # Open points dataset
    points_ds = ogr.Open(points_path)
    points_layer = points_ds.GetLayer()

    # Get raster band
    band = raster_ds.GetRasterBand(1)

    # Loop through points and extract values
    for point_feature in points_layer:
        point_geom = point_feature.GetGeometryRef()
        x, y = point_geom.GetX(), point_geom.GetY()

        # Convert coordinates to pixel coordinates
        pixel_x, pixel_y = int((x - raster_ds.GetGeoTransform()[0]) / raster_ds.GetGeoTransform()[1]), \
                           int((y - raster_ds.GetGeoTransform()[3]) / raster_ds.GetGeoTransform()[5])

        # Read raster value at the given pixel
        value = band.ReadAsArray(pixel_x, pixel_y, 1, 1)[0][0]

        print(f"Point ({x}, {y}): Raster Value = {value}")

if __name__ == "__main__":
    raster_path = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\Vindefjallen_testing\Arc_Vindefjellen\temp\FocalSt_Slope_E1.tif"
    points_path = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\Vindefjallen_testing\Arc_Vindefjellen\temp\slc_granule_every_point_sweref_XYTableToPoint1.shp"

    extract_values_from_raster(raster_path, points_path)

Point (529649.2181000002, 7349834.175000001): Raster Value = 3.8383514881134033
Point (534661.4522000002, 7309911.059): Raster Value = 2.076606035232544
Point (531291.7264999999, 7353479.210000001): Raster Value = 3.671757221221924
Point (534712.8109999998, 7305912.951000001): Raster Value = 4.158007621765137
Point (533627.8268999998, 7350234.324000001): Raster Value = 6.097835540771484
Point (531224.7772000004, 7307867.528000001): Raster Value = 3.6996469497680664
Point (551564.2797999997, 7332621.634): Raster Value = 7.713944435119629
Point (522072.97250000015, 7355934.5929999985): Raster Value = 8.954695701599121
Point (531257.926, 7341357.539999999): Raster Value = 2.3543894290924072
Point (538235.3079000004, 7342946.921999998): Raster Value = 1.116188883781433
Point (547089.1167000001, 7338562.269000001): Raster Value = 2.6447904109954834
Point (551564.2797999997, 7332621.634): Raster Value = 7.713944435119629
Point (526051.5524000004, 7356334.761999998): Raster Value = 6.31822061

In [9]:
raster_path = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\Vindefjallen_testing\Arc_Vindefjellen\temp\FocalSt_Slope_E1.tif"
points_path = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\Vindefjallen_testing\Arc_Vindefjellen\temp\slc_granule_every_point_sweref_XYTableToPoint1.shp"

raster_ds = gdal.Open(raster_path)
print(raster_ds)
# Open points dataset
points_ds = ogr.Open(points_path)
points_layer = points_ds.GetLayer()

# Get raster band
band = raster_ds.GetRasterBand(1)



<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000018B0D4DDC00> >


In [11]:
raster_ds.GetProjection()

'PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWEREF99",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6619"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4619"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3006"]]'

# Rasterstats

https://github.com/perrygeo/python-rasterstats?tab=readme-ov-file

In [10]:
# %pip install rasterstats
from rasterstats import zonal_stats, point_query
import fiona

shp_src = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\ValueToPoint\shp_files\jorddjup_buffer1.shp"
raster_src = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\ValueToPoint\raster\jorddjup_10x10m.tif"


In [8]:
with fiona.open(shp_src, "r") as shapefile:
    features = [feature["geometry"] for feature in shapefile]
features    

In [15]:
with fiona.open(shp_src) as src:
    zs = zonal_stats(src, raster_src, stats=['min', 'max', 'median', 'majority', 'sum'])

zs

[{'min': 14.0, 'max': 16.0, 'sum': 44.0, 'median': 14.0, 'majority': 14.0},
 {'min': 6.0, 'max': 7.0, 'sum': 27.0, 'median': 7.0, 'majority': 7.0},
 {'min': 33.0, 'max': 33.0, 'sum': 66.0, 'median': 33.0, 'majority': 33.0},
 {'min': 0.0, 'max': 0.0, 'sum': 0.0, 'median': 0.0, 'majority': 0.0}]

# Clip rasters

In [3]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import box

# File paths
raster_file = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\ValueToPoint\raster\jorddjup_10x10m_croppedsqaure.tif"
shapefile = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\ValueToPoint\shp_files\study_area.shp"
output_clipped_raster = r'C:\Users\tryggvisi\Documents\my-awesome-masters-project\Testing\ValueToPoint\clip\clipped_raster_square.tif'

# Read the shapefile
gdf = gpd.read_file(shapefile)

# Read the raster data
with rasterio.open(raster_file) as src:
    # Reproject the shapefile to the raster's CRS if they are different
    gdf = gdf.to_crs(src.crs)
    
    # Get the geometry of the shapefile
    geom = gdf.geometry.values[0]
    
    # Clip the raster with the shapefile's geometry
    out_image, out_transform = mask(src, [geom], crop=True)
    out_meta = src.meta.copy()

    # Update metadata for the clipped raster
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    # Write the clipped raster to a new file
    with rasterio.open(output_clipped_raster, "w", **out_meta) as dest:
        dest.write(out_image)


In [ ]:
processing.run("grass7:v.sample", {'input':'C:/Users/tryggvisi/Documents/my-awesome-masters-project/Testing/ValueToPoint/shp_files/jorddjup_points_3006.shp','column':'FID','raster':'C:/Users/tryggvisi/Documents/my-awesome-masters-project/Data/Landcover/Jorddjupsmodell_latest_shp_tif_br_Depth_of_earth_97bb23f7-a186-46f9-9599-1f11ded493d7_/jorddjup_10x10m/jorddjup_10x10m.tif','zscale':1,'method':1,'output':'TEMPORARY_OUTPUT','GRASS_REGION_PARAMETER':None,'GRASS_REGION_CELLSIZE_PARAMETER':0,'GRASS_SNAP_TOLERANCE_PARAMETER':-1,'GRASS_MIN_AREA_PARAMETER':0.0001,'GRASS_OUTPUT_TYPE_PARAMETER':0,'GRASS_VECTOR_DSCO':'','GRASS_VECTOR_LCO':'','GRASS_VECTOR_EXPORT_NOCAT':False})